# Anthropic Commodities Analysis Agent with Real-time News Access (built with LlamaIndex)

## Overview <br>
This system provides a comprehensive toolkit for analyzing commodities market data using the Financial Modeling Prep (FMP) API. It's designed to work within Google Colab notebooks and provides real-time commodities market analysis capabilities.

### Tools used:


1.   Anthropic API - Claude 3.5 Sonnett   
2.   LlamaIndex for orchestration
3.   Financial Modeling Prep API for commodities information
4.   News API to create up-to-date view on what's influencing commodities markets



In [1]:
!pip install llama-index-llms-anthropic -q
!pip install llama-index -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.8/176.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.8 MB/s eta 0:00:00


In [2]:
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

import nest_asyncio

nest_asyncio.apply()

## 2. Instantiate Anthropic and News API Keys

Step 1: Instantiate Baseline LLM to be used


*   Experiment 1: Use Claude Sonnet for tool use, code generation and parsing
*   Experiment 2: Use OpenAI o1 models to test reasoning capabilities



In [3]:
from google.colab import userdata

CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')
OAI_API_KEY = userdata.get('OPENAI_API_KEY')
FMP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')
NEWS_API_KEY = userdata.get('NEWS_API_KEY')

In [4]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=NEWS_API_KEY)

ModuleNotFoundError: No module named 'newsapi'

In [5]:
openai_llm = OpenAI(model="gpt-4o-mini", api_key=OAI_API_KEY)

In [6]:
anthropic_llm = Anthropic(model="claude-3-5-sonnet-20240620", api_key=CLAUDE_API_KEY)

Step 2: Instantiate News API for Retrieval

## 3. Define Classes and Tools for LLM Use

### 3.0 Core Components

#### CommoditiesTracker Class
The main class that handles data fetching and analysis of commodities market data.


##### Key Methods:

###### `__init__()`
* Initializes the tracker with FMP API credentials from Colab userdata
* Sets up the base URL for API calls
* Triggers initial data fetch

###### `_fetch_data()`
* Private method that retrieves fresh commodities data from FMP API
* Converts raw JSON data into a pandas DataFrame
* Includes error handling for API requests

###### `refresh_data()`
* Public method to manually update data from the API
* Useful for getting latest market updates
* Calls `_fetch_data()` internally to refresh the DataFrame

###### `get_commodity_info(symbol: Optional[str] = None, name: Optional[str] = None)`
* Retrieves detailed information for a specific commodity
* Can search by symbol (e.g., 'GCUSD') or name (e.g., 'Gold')
* Returns comprehensive commodity data including:
  * Current price and changes
  * Daily trading ranges
  * Volume information
  * Historical averages

###### `get_top_movers(n: int = 5, by: str = 'changesPercentage')`
* Identifies top-performing commodities
* Customizable number of results (default: 5)
* Sorting metrics available:
  * Price change percentage
  * Trading volume
  * Current price
* Returns formatted list of top performers with key metrics

###### `get_market_summary()`
* Provides overall market statistics including:
  * Total number of tracked commodities
  * Count of commodities trending up/down
  * Most actively traded commodity
  * Biggest gainer of the session
  * Biggest loser of the session
* Returns structured dictionary of market metrics

###### `get_commodity_analysis(symbol: str)`
* Performs technical analysis for a specific commodity
* Technical indicators included:
  * 50-day moving average comparison
  * 200-day moving average comparison
  * Daily price range analysis
  * Yearly price range analysis
  * Volume analysis vs average
  * Price change metrics
* Returns detailed technical analysis dictionary


### 3.1 Initialize Commodities Tracker Class

In [7]:
from google.colab import userdata
import requests
import pandas as pd
from typing import Dict, List, Optional, Union
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

class CommoditiesTracker:
    def __init__(self):
        """
        Initialize the CommoditiesTracker using Google Colab userdata for API key.
        """
        self.api_key = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')
        if not self.api_key:
            raise ValueError("FMP API key not found in Colab userdata")

        self.base_url = "https://financialmodelingprep.com/api/v3"
        self.df = None
        self._fetch_data()

    def _fetch_data(self) -> None:
        """
        Fetch fresh data from the FMP API and update the internal dataframe.
        """
        url = f"{self.base_url}/quotes/commodity?apikey={self.api_key}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            self.raw_data = response.json()
            self.df = pd.DataFrame(self.raw_data)
        except requests.exceptions.RequestException as e:
            raise Exception(f"Failed to fetch commodities data: {str(e)}")
        except Exception as e:
            raise Exception(f"Error processing commodities data: {str(e)}")

    def refresh_data(self) -> None:
        """
        Manually refresh the data from the API.
        """
        self._fetch_data()

    def get_commodity_info(self, symbol: Optional[str] = None, name: Optional[str] = None) -> Dict:
        """
        Get detailed information for a specific commodity by symbol or name.
        """
        try:
            if symbol:
                commodity = self.df[self.df['symbol'] == symbol].to_dict('records')
            elif name:
                commodity = self.df[self.df['name'].str.contains(name, case=False, na=False)].to_dict('records')
            else:
                return {"error": "Must provide either symbol or name"}

            if not commodity:
                return {"error": f"Commodity not found for symbol: {symbol} or name: {name}"}

            return commodity[0]
        except Exception as e:
            return {"error": f"Error retrieving commodity information: {str(e)}"}

    def get_top_movers(self, n: int = 5, by: str = 'changesPercentage') -> List[Dict]:
        """
        Get top n commodities by specified metric.
        """
        try:
            sorted_df = self.df.nlargest(n, by)
            return sorted_df[['symbol', 'name', by, 'price', 'change']].to_dict('records')
        except Exception as e:
            return [{"error": f"Error retrieving top movers: {str(e)}"}]

    def get_market_summary(self) -> Dict:
        """
        Get overall market summary statistics.
        """
        try:
            return {
                "total_commodities": len(self.df),
                "commodities_up": len(self.df[self.df['change'] > 0]),
                "commodities_down": len(self.df[self.df['change'] < 0]),
                "most_active": self.df.nlargest(1, 'volume')[['symbol', 'name', 'volume', 'price']].to_dict('records')[0],
                "biggest_gainer": self.df.nlargest(1, 'changesPercentage')[['symbol', 'name', 'changesPercentage', 'price']].to_dict('records')[0],
                "biggest_loser": self.df.nsmallest(1, 'changesPercentage')[['symbol', 'name', 'changesPercentage', 'price']].to_dict('records')[0]
            }
        except Exception as e:
            return {"error": f"Error retrieving market summary: {str(e)}"}

    def get_commodity_analysis(self, symbol: str) -> Dict:
        """
        Get detailed technical analysis for a specific commodity.
        """
        try:
            commodity = self.df[self.df['symbol'] == symbol].iloc[0]

            ma50_position = "above" if commodity['price'] > commodity['priceAvg50'] else "below"
            ma200_position = "above" if commodity['price'] > commodity['priceAvg200'] else "below"

            return {
                "symbol": commodity['symbol'],
                "name": commodity['name'],
                "current_price": commodity['price'],
                "technical_analysis": {
                    "ma50_analysis": f"Price is {ma50_position} 50-day MA ({commodity['priceAvg50']:.2f})",
                    "ma200_analysis": f"Price is {ma200_position} 200-day MA ({commodity['priceAvg200']:.2f})",
                    "day_range": f"${commodity['dayLow']:.2f} - ${commodity['dayHigh']:.2f}",
                    "year_range": f"${commodity['yearLow']:.2f} - ${commodity['yearHigh']:.2f}",
                    "volume_analysis": f"Current volume ({commodity['volume']}) vs Avg volume ({commodity['avgVolume']})",
                    "price_change": {
                        "value": commodity['change'],
                        "percentage": commodity['changesPercentage']
                    }
                }
            }
        except Exception as e:
            return {"error": f"Error retrieving commodity analysis: {str(e)}"}


### 3.2 Helper Functions for Tool Use

- Users can create multiple helper functions for all forms of analysis - other implementations could include sentiment analysis, dataframe creation, visualizations tools, etc


In [8]:
# Function wrappers for the LLM tools
def get_commodity_info(symbol: Optional[str] = None, name: Optional[str] = None) -> Dict:
    """
    Get detailed information for a specific commodity by symbol or name.

    Args:
        symbol (str, optional): Trading symbol of the commodity (e.g., 'GCUSD' for Gold)
        name (str, optional): Name of the commodity (e.g., 'Gold')
    """
    try:
        tracker = CommoditiesTracker()
        return tracker.get_commodity_info(symbol, name)
    except Exception as e:
        return {"error": f"Failed to get commodity info: {str(e)}"}

def get_top_movers(n: int = 5, metric: str = 'changesPercentage') -> List[Dict]:
    """
    Get top performing commodities by specified metric.

    Args:
        n (int): Number of commodities to return (default: 5)
        metric (str): Metric to sort by - options: 'changesPercentage', 'volume', 'price'
    """
    try:
        tracker = CommoditiesTracker()
        return tracker.get_top_movers(n, metric)
    except Exception as e:
        return [{"error": f"Failed to get top movers: {str(e)}"}]

def get_market_summary() -> Dict:
    """
    Get overall commodities market summary statistics.
    """
    try:
        tracker = CommoditiesTracker()
        return tracker.get_market_summary()
    except Exception as e:
        return {"error": f"Failed to get market summary: {str(e)}"}

def get_commodity_analysis(symbol: str) -> Dict:
    """
    Get detailed technical analysis for a specific commodity.

    Args:
        symbol (str): Trading symbol of the commodity (e.g., 'GCUSD' for Gold)
    """
    try:
        tracker = CommoditiesTracker()
        return tracker.get_commodity_analysis(symbol)
    except Exception as e:
        return {"error": f"Failed to get commodity analysis: {str(e)}"}

## 4. Create Agentic Flow

### 4.1 Convert Functions to Tools for LLM Use

In [9]:
commodity_info_tool = FunctionTool.from_defaults(fn=get_commodity_info)
top_movers_tool = FunctionTool.from_defaults(fn=get_top_movers)
market_summary_tool = FunctionTool.from_defaults(fn=get_market_summary)
commodity_analysis_tool = FunctionTool.from_defaults(fn=get_commodity_analysis)


### 4.2 Create an Anthropic Claude Agent

In [10]:
claude_agent = FunctionCallingAgent.from_tools(
    [
        commodity_info_tool,
        top_movers_tool,
        market_summary_tool,
        commodity_analysis_tool
    ],
    llm=anthropic_llm,
    verbose=False,
    allow_parallel_tool_calls=False,
)

# Chat with an Agent

In [11]:
query= "Tell me more about the Cotton prices and how is tredning compared to 52-week highs/lows and the moving averages?"
response = claude_agent.chat(query)
print(str(response))

Great! Now I have the information about Cotton prices and trends. Let me break it down for you:

1. Current Price: The current price of Cotton is $71.04 per unit.

2. Daily Change: Cotton is down 1.44% today, with a decrease of $1.04.

3. 52-Week Range:
   - 52-Week High: $107.25
   - 52-Week Low: $65.30
   - Current price ($71.04) is closer to the 52-week low than the high, indicating a generally bearish trend over the past year.

4. Moving Averages:
   - 50-Day Moving Average: $71.0772
   - 200-Day Moving Average: $77.6253

5. Trend Analysis:
   - The current price ($71.04) is slightly below the 50-day moving average ($71.0772), suggesting a short-term bearish trend.
   - The current price is significantly below the 200-day moving average ($77.6253), indicating a longer-term bearish trend.
   - The 50-day moving average is below the 200-day moving average, which is typically considered a bearish signal (often called a "death cross").

6. Daily Range:
   - Today's Low: $70.76
   - Tod

In [12]:
query= "Which commodities have been trending favorably over the past 3-6 months?"
response = claude_agent.chat(query)
print(str(response))

Based on the data provided by the "get_top_movers" function, we can see the commodities that have been performing well recently. However, it's important to note that this data shows the most recent changes, which might not fully reflect the 3-6 month trend you're asking about. Nonetheless, these commodities are showing positive momentum, which could be indicative of favorable trends over a longer period.

Here are the top performing commodities based on recent percentage changes:

1. Cocoa (CCUSD): +9.58%
2. Coffee (KCUSX): +4.47%
3. Copper (HGUSD): +4.40%
4. Aluminum Futures (ALIUSD): +3.92%
5. Soybean Oil Futures (ZLUSX): +3.87%
6. Silver Futures (SIUSD and SILUSD): +1.88%
7. Feeder Cattle Futures (GFUSX): +1.45%
8. Nasdaq 100 (NQUSD): +1.44% (Note: This is an index, not a traditional commodity)
9. Crude Oil (CLUSD): +1.19%

To get a more accurate picture of the 3-6 month trends, we would need historical data for these commodities. However, based on this recent performance, we can ma

In [13]:
query= "Which metal commodities have been trending favorably? Present your findings in a table format"
response = claude_agent.chat(query)
print(str(response))

Now that we have the data, let's filter out the metal commodities and present them in a table format. The metal commodities in this list are Copper, Aluminum, Silver, Gold, and Platinum.

Here's a table of the metal commodities that have been trending favorably:

| Symbol | Name                 | Price   | Change | Change % |
|--------|----------------------|---------|--------|----------|
| HGUSD  | Copper               | $4.4285 | $0.1825| +4.30%   |
| ALIUSD | Aluminum Futures     | $2697.25| $101.75| +3.92%   |
| SIUSD  | Silver Futures       | $31.925 | $0.594 | +1.90%   |
| GCUSD  | Gold Futures         | $2706.8 | $30.5  | +1.14%   |
| PLUSD  | Platinum             | $998.8  | $6.0   | +0.60%   |

Observations from this data:

1. Copper is showing the strongest positive trend among metal commodities, with a significant increase of 4.30%.

2. Aluminum is also performing very well, with a notable increase of 3.92%.

3. Silver is showing moderate gains at 1.90%, which is still a pos

In [ ]:
query= "Compare gold and silver commodities - do fundamental analysis?"
response = claude_agent.chat(query)
print(str(response))

Now, let's compare gold and silver commodities based on this information:

1. Price and Performance:
   - Gold: $2,756.80 per ounce, up 0.27% today
   - Silver: $32.86 per ounce, up 0.20% today

   Both metals are showing positive performance today, with gold slightly outperforming silver.

2. Year-to-Date Performance:
   - Gold: Currently near its yearly high of $2,789.00 (52-week range: $1,932.60 - $2,789.00)
   - Silver: Also near its yearly high of $34.84 (52-week range: $21.93 - $34.84)

   Both metals have shown strong performance over the past year, with current prices near their yearly highs.

3. Moving Averages:
   - Gold: Price is above both 50-day MA ($2,616.87) and 200-day MA ($2,357.10)
   - Silver: Price is above both 50-day MA ($31.07) and 200-day MA ($28.07)

   Both metals are trading above their key moving averages, indicating bullish trends.

4. Volume:
   - Gold: Current volume (50,906) is significantly higher than average volume (614)
   - Silver: Current volume (1

In [ ]:
query= "Create a fundamental analysis on Platinum"
response = claude_agent.chat(query)
print(str(response))

Based on this information, let's create a fundamental analysis for Platinum:

Fundamental Analysis of Platinum (PLUSD)

1. Price and Performance:
   - Current Price: $1,010.40 per ounce
   - Daily Change: Up 1.08% ($10.80)
   - Year Range: $838.60 - $1,084.60
   - The current price is closer to the yearly high, indicating a generally positive trend over the past year.

2. Technical Indicators:
   - Moving Averages: The price is above both the 50-day ($979.43) and 200-day ($958.30) moving averages, suggesting a bullish trend in both short and long-term timeframes.
   - Volume: Current volume (6,172) is significantly higher than the average volume (215), indicating increased interest and trading activity.

3. Supply and Demand Factors:
   a. Industrial Demand:
      - Platinum has significant industrial applications, particularly in the automotive industry for catalytic converters.
      - It's also used in jewelry, electronics, and chemical processing.
      - The price increase might s